In [1]:
import whisperx
import gc
#!pip install jiwer
from jiwer import wer, cer
import torch
import soundfile as sf
from transformers import WhisperProcessor, WhisperForConditionalGeneration
from peft import PeftModel, LoraConfig
import time
import evaluate
import random
import os
import re
from IPython.display import Audio
import string

d:\anaconda3\envs\whisperx\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
INFO:speechbrain.utils.quirks:Applied quirks (see `speechbrain.utils.quirks`): [disable_jit_profiling, allow_tf32]
INFO:speechbrain.utils.quirks:Excluded quirks specified by the `SB_DISABLE_QUIRKS` environment (comma-separated list): []
d:\anaconda3\envs\whisperx\lib\site-packages\pyannote\audio\pipelines\speaker_verification.py:45: UserWarning: Module 'speechbrain.pretrained' was deprecated, redirecting to 'speechbrain.inference'. Please update your script. This is a change from SpeechBrain 1.0. See: https://github.com/speechbrain/speechbrain/releases/tag/v1.0.0
  from speechbrain.pretrained import (
INFO:datasets:PyTorch version 2.0.0 available.


In [2]:
metric = evaluate.load("wer")

def clean_text(text):
    # Remove non-ASCII characters
    text = re.sub(r'[^\x00-\x7F]+', ' ', text)
    # Remove punctuation except apostrophes
    text = re.sub(r'[{}]+'.format(re.escape(string.punctuation.replace("'", ""))), ' ', text)
    # Remove extra spaces
    text = re.sub(r'\s+', ' ', text).strip()
    return text

def compute_metrics(pred_str, label_str):
    wer = 100 * metric.compute(predictions=pred_str, references=label_str)
    return {"wer": wer}

In [3]:
device = "cuda" 
batch_size = 16 # reduce if low on GPU mem
compute_type = "float16" # change to "int8" if low on GPU mem (may reduce accuracy)

# (Converting finetuned Whisper to WhisperX support)
not needed since WhisperX cannot load weights from fine-tuned model yet

In [201]:
import json

# Define paths to your existing files
adapter_config_path = "output_dir/checkpoint-903/adapter_config.json"
preprocessor_config_path = "output_dir/checkpoint-903/preprocessor_config.json"
tokenizer_config_path = "output_dir/checkpoint-903/tokenizer_config.json"
output_config_path = "output_dir/checkpoint-903/config.json"

# Load the existing files
with open(adapter_config_path, "r") as f:
    adapter_config = json.load(f)

with open(preprocessor_config_path, "r") as f:
    preprocessor_config = json.load(f)

with open(tokenizer_config_path, "r") as f:
    tokenizer_config = json.load(f)

# Combine into the final config.json structure
config = {
    "_name_or_path": "openai/whisper-small",
    "architectures": ["WhisperForConditionalGeneration"],
    "d_model": 512,
    "decoder_attention_heads": 8,
    "decoder_ffn_dim": 2048,
    "decoder_layers": 6,
    "encoder_attention_heads": 8,
    "encoder_ffn_dim": 2048,
    "encoder_layers": 6,
    "model_type": "whisper",
    "use_cache": True,
    "task_specific_params": {
        "transcribe": {"max_length": 448},
        "translate": {"max_length": 512}
    },
    "vocab_size": 51864,
    "num_hidden_layers": 6,
    "tie_word_embeddings": False,
    "adapter_config": adapter_config,
    "preprocessor_config": preprocessor_config,
    "tokenizer_config": tokenizer_config
}

# Save the config.json file
with open(output_config_path, "w") as f:
    json.dump(config, f, indent=4)

print(f"config.json recreated and saved to {output_config_path}")


config.json recreated and saved to output_dir/checkpoint-903/config.json


In [107]:
# we need to get tokenizer_config.json and merges.txt
!wget -O output_dir/checkpoint-903/merges.txt https://huggingface.co/openai/whisper-small/blob/main/tokenizer_config.json
!wget -O output_dir/checkpoint-903/merges.txt https://huggingface.co/openai/whisper-small/resolve/main/merges.txt

# transfor model to whisperx
!ct2-transformers-converter --model output_dir/checkpoint-903 --output_dir output_dir/whisperx --quantization float16

config.json created at output_dir/checkpoint-903\config.json
Model conversion complete.
config.json created at output_dir/checkpoint-903\config.json
Model conversion complete.


# Loading Models

In [4]:
def load_whisper_model():
    whisper_model_name = "openai/whisper-small"
    whisper_model = WhisperForConditionalGeneration.from_pretrained(whisper_model_name)
    whisper_processor = WhisperProcessor.from_pretrained(whisper_model_name, language="en", task="transcribe")
    return whisper_model, whisper_processor

# Load the fine-tuned Whisper model
def load_fine_tuned_model():
    whisper_model_name = "openai/whisper-small"
    adapter_path = "ashkab/whisper_fine-tuned"
    config = LoraConfig(r=32, lora_alpha=64, target_modules=["q_proj", "v_proj"], lora_dropout=0.5, bias="none")
    base_model = WhisperForConditionalGeneration.from_pretrained(whisper_model_name)
    fine_tuned_model = PeftModel.from_pretrained(base_model, adapter_path)
    fine_tuned_processor = WhisperProcessor.from_pretrained(whisper_model_name, language="en", task="transcribe")
    return fine_tuned_model, fine_tuned_processor

# Load WhisperX model
def load_whisperx_model(device, compute_type):
    return whisperx.load_model("small", device, compute_type=compute_type, language="en")


def load_fine_tuned_whisperx_model(device, compute_type, fine_tuned_model_dir):
    """
    Load WhisperX with a fine-tuned CTranslate2 Whisper model.
    """
    # Load the base WhisperX model
    whisperx_model = whisperx.load_model("small", device, compute_type=compute_type, language="en")

    # Ensure the model is loaded as a CTranslate2 translator
    from ctranslate2 import Translator
    translator = Translator(fine_tuned_model_dir, device=device)

    # Replace the WhisperX encoder with the fine-tuned translator
    whisperx_model.hf_encoder = translator

    return whisperx_model

In [5]:
# Transcribe with Whisper or fine-tuned Whisper
def transcribe_with_model(model, processor, audio_path, chunk_size=30):
    """
    Transcribe audio using the specified model and processor, supporting audio longer than chunk_size seconds.
    """
    audio_input, sample_rate = sf.read(audio_path)
    chunk_samples = int(chunk_size * sample_rate)
    transcriptions = []
    for start_sample in range(0, len(audio_input), chunk_samples):
        end_sample = start_sample + chunk_samples
        audio_chunk = audio_input[start_sample:end_sample]
        input_features = processor.feature_extractor(
            audio_chunk, sampling_rate=sample_rate, return_tensors="pt"
        ).input_features
        with torch.no_grad():
            generated_ids = model.generate(input_features)
        transcription = processor.batch_decode(generated_ids, skip_special_tokens=True)[0]
        transcriptions.append(transcription)
    return " ".join(transcriptions)

# Transcribe with WhisperX
def transcribe_with_whisperx(whisperx_model, audio_path, batch_size=16):
    audio = whisperx.load_audio(audio_path)
    result = whisperx_model.transcribe(audio, batch_size=batch_size)
    return result["segments"]

# Separate transcription functions for each model
def transcribe_with_base_whisper(audio_path):
    whisper_model, whisper_processor = load_whisper_model()
    print("Transcribing with base Whisper model...")
    start_time = time.time()
    transcription = transcribe_with_model(whisper_model, whisper_processor, audio_path)
    time_needed = time.time() - start_time
    print(f"Base Whisper transcription completed in {time.time() - start_time:.2f} seconds")
    return transcription, time_needed

def transcribe_with_fine_tuned_whisper(audio_path):
    fine_tuned_model, fine_tuned_processor = load_fine_tuned_model()
    print("Transcribing with fine-tuned Whisper model...")
    start_time = time.time()
    transcription = transcribe_with_model(fine_tuned_model, fine_tuned_processor, audio_path)
    time_needed = time.time() - start_time
    print(f"Fine-tuned Whisper transcription completed in {time.time() - start_time:.2f} seconds")
    return transcription, time_needed

def transcribe_with_whisperx_model(audio_path):
    device = "cuda" if torch.cuda.is_available() else "cpu"
    compute_type = "float16" if device == "cuda" else "int8"
    whisperx_model = load_whisperx_model(device, compute_type)
    print("Transcribing with WhisperX model...")
    start_time = time.time()
    transcription = transcribe_with_whisperx(whisperx_model, audio_path)
    time_needed = time.time() - start_time
    print(f"WhisperX transcription completed in {time.time() - start_time:.2f} seconds")
    return transcription, time_needed

def transcribe_with_fine_tuned_whisperx(audio_path):
    """
    Transcribe audio using the fine-tuned WhisperX model.
    """
    device = "cuda" if torch.cuda.is_available() else "cpu"
    compute_type = "float16" if device == "cuda" else "int8"

    # Specify the directory of the transformed fine-tuned model
    fine_tuned_model_dir = "output_dir/checkpoint-903"  # Directory where the transformed model is saved

    # Load the fine-tuned WhisperX model
    fine_tuned_whisperx_model = load_fine_tuned_whisperx_model(device, compute_type, fine_tuned_model_dir)

    print("Transcribing with fine-tuned WhisperX model...")
    start_time = time.time()
    transcription = transcribe_with_whisperx(fine_tuned_whisperx_model, audio_path)
    time_needed = time.time() - start_time
    print(f"Fine-tuned WhisperX transcription completed in {time.time() - start_time:.2f} seconds")

    return transcription, time_needed

# Evaluate on long audio

In [6]:
# specify on what you rather want to test - short or long
audio_file = "data/audio_files/20190130_FOMC.wav" # path to audio file
text_file = "data/transcript_files/20190130_FOMC.txt" # path to text file

In [ ]:
with open(text_file, "r") as file:
    true_transcription = file.read().replace("\n", " ")

print("True transcription:\n", true_transcription)

# Transcribe with base Whisper
base_whisper_transcription = transcribe_with_base_whisper(audio_file)[0]
print("Base Whisper Transcription:")
print(base_whisper_transcription, "\n", compute_metrics([clean_text(base_whisper_transcription)], [clean_text(true_transcription)]))

# Transcribe with fine-tuned Whisper
fine_tuned_whisper_transcription = transcribe_with_fine_tuned_whisper(audio_file)[0]
print("Fine-Tuned Whisper Transcription:")
print(fine_tuned_whisper_transcription, "\n", compute_metrics([clean_text(fine_tuned_whisper_transcription)], [clean_text(true_transcription)]))

# Transcribe with WhisperX
whisperx_transcription = transcribe_with_whisperx_model(audio_file)[0]
print("WhisperX Transcription:", "".join([i["text"] for i in whisperx_transcription]), "\n", compute_metrics([clean_text("".join([i["text"] for i in whisperx_transcription]))], [clean_text(true_transcription)]))

# Transcribe with fine-tuned WhisperX
#fine_tuned_whisperx_transcription = transcribe_with_fine_tuned_whisperx(audio_file)
#print("Fine-Tuned WhisperX Transcription:", "".join([i["text"] for i in fine_tuned_whisperx_transcription]), "\n", compute_metrics(clean_text("".join([i["text"] for i in fine_tuned_whisperx_transcription])), clean_text(true_transcription)))

True transcription:
 Good afternoon, everyone, and welcome. I will start with a recap of our discussions, including our assessment of the outlook for the economy and the judgments we made about our interest rate policy and our balance sheet. I will cover the decisions we made today as well as our ongoing discussions of matters on which we expect to make decisions in coming meetings. My colleagues and I have one overarching goal: to sustain the economic expansion, with a strong job market and stable prices, for the benefit of the American people. The U. economy is in a good place, and we will continue to use our monetary policy tools to help keep it there. The jobs picture continues to be strong, with the unemployment rate near historic lows and with stronger wage gains. Inflation remains near our 2 percent goal. We continue to expect that the American economy will grow at a solid pace in 2019, although likely slower than the very strong pace of 2018. We believe that our current policy 

Lightning automatically upgraded your loaded checkpoint from v1.5.4 to v2.3.3. To apply the upgrade to your files permanently, run `python -m pytorch_lightning.utilities.upgrade_checkpoint C:\Users\Artem Shkabruk\.cache\torch\whisperx-vad-segmentation.bin`


Model was trained with pyannote.audio 0.0.1, yours is 3.1.1. Bad things might happen unless you revert pyannote.audio to 0.x.
Model was trained with torch 1.10.0+cu102, yours is 2.0.0. Bad things might happen unless you revert torch to 1.x.
Transcribing with WhisperX model...


d:\anaconda3\envs\whisperx\lib\site-packages\pyannote\audio\utils\reproducibility.py:74: ReproducibilityWarning: TensorFloat-32 (TF32) has been disabled as it might lead to reproducibility issues and lower accuracy.
It can be re-enabled by calling
   >>> import torch
   >>> torch.backends.cuda.matmul.allow_tf32 = True
   >>> torch.backends.cudnn.allow_tf32 = True
See https://github.com/pyannote/pyannote-audio/issues/1370 for more details.

  warnings.warn(


WhisperX transcription completed in 22.35 seconds
WhisperX Transcription:  Good afternoon, everyone, and welcome. I will start with a recap of our discussions, including our assessment of the outlook for the economy and the judgments we made about our interest rate policy and our balance sheet. I will cover the decisions we made today, as well as our ongoing discussions of matters on which we expect to make decisions in coming meetings. My colleagues and I have one overarching goal to sustain the economic expansion with a strong job market and stable prices for the benefit of the American people. The U.S. economy is in a good place and we will continue to use our monetary policy tools to help keep it there. The jobs picture continues to be strong with the unemployment rate near historic lows and with stronger wage gains. Inflation remains near our 2 percent goal. We continue to expect that the American economy will grow at a solid pace in 2019, although likely slower than the very stro

# Evaluate on random short audio

In [206]:
short_audio_dir = "data/split_audio"
short_text_dir = "data/split_text"

In [207]:
num = 100

wer_rate_base = []
wer_rate_finetuned = []
wer_rate_whisperx = []

time_base = []
time_finetuned = []
time_whisperx = []

for i in range(num):
    random_wav = random.choice(os.listdir(short_audio_dir)[5001:]) # since model has been trained on first 5000 files
    random_wav_path = os.path.join(short_audio_dir, random_wav)

    # Print the corresponding text file content for manual inspection
    with open(os.path.join(short_text_dir, random_wav[:-4] + ".txt"), "r") as txt:
        original_transcription = txt.read()
        
        original_transcription = clean_text(original_transcription)
        if len(original_transcription.split(" ")) < 7: # only transcribe files with more than 7 words
            continue
        
        print("Original Transcription:\n", original_transcription, "\n")

        # Transcribe with base Whisper
        base_whisper_transcription = transcribe_with_base_whisper(random_wav_path)[0]
        print("Base Whisper Transcription:")
        print(base_whisper_transcription)

        # Transcribe with fine-tuned Whisper
        fine_tuned_whisper_transcription = transcribe_with_fine_tuned_whisper(random_wav_path)[0]
        print("Fine-Tuned Whisper Transcription:")
        print(fine_tuned_whisper_transcription)

        # Transcribe with WhisperX
        whisperx_transcription = transcribe_with_whisperx_model(random_wav_path)[0]
        whisperx_transcription = "".join([i["text"] for i in whisperx_transcription])
        print("WhisperX Transcription:\n", whisperx_transcription)

        # Transcribe with fine-tuned WhisperX
        #fine_tuned_whisperx_transcription = transcribe_with_fine_tuned_whisperx(random_wav_path)
        #print("Fine-Tuned WhisperX Transcription:", "".join([i["text"] for i in fine_tuned_whisperx_transcription]))

        wer_rate_base.append(compute_metrics([clean_text(base_whisper_transcription)], [clean_text(original_transcription)])["wer"])
        wer_rate_finetuned.append(compute_metrics([clean_text(fine_tuned_whisper_transcription)], [clean_text(original_transcription)])["wer"])
        wer_rate_whisperx.append(compute_metrics([clean_text(whisperx_transcription)], [clean_text(original_transcription)])["wer"])

        time_base.append(transcribe_with_base_whisper(random_wav_path)[1])
        time_finetuned.append(transcribe_with_fine_tuned_whisper(random_wav_path)[1])
        time_whisperx.append(transcribe_with_whisperx_model(random_wav_path)[1])

                

Original Transcription:
 a broad expectation certainly not the depression forecast 



d:\anaconda3\envs\whisperx\lib\site-packages\huggingface_hub\file_download.py:797: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Transcribing with base Whisper model...
Base Whisper transcription completed in 5.80 seconds
Base Whisper Transcription:
 a broad expectation, certainly not the depression forecast.


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Transcribing with fine-tuned Whisper model...
Fine-tuned Whisper transcription completed in 5.88 seconds
Fine-Tuned Whisper Transcription:
a broad expectation certainly not the depression forecast


...

Transcribing with base Whisper model...
Base Whisper transcription completed in 9.06 seconds


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Transcribing with fine-tuned Whisper model...
Fine-tuned Whisper transcription completed in 7.71 seconds


Lightning automatically upgraded your loaded checkpoint from v1.5.4 to v2.3.3. To apply the upgrade to your files permanently, run `python -m pytorch_lightning.utilities.upgrade_checkpoint C:\Users\Artem Shkabruk\.cache\torch\whisperx-vad-segmentation.bin`


Model was trained with pyannote.audio 0.0.1, yours is 3.1.1. Bad things might happen unless you revert pyannote.audio to 0.x.
Model was trained with torch 1.10.0+cu102, yours is 2.0.0. Bad things might happen unless you revert torch to 1.x.
Transcribing with WhisperX model...
WhisperX transcription completed in 0.44 seconds


In [208]:
import numpy as np

print(np.mean(wer_rate_base))
print(np.mean(wer_rate_finetuned))
print(np.mean(wer_rate_whisperx))

print(np.mean(time_base))
print(np.mean(time_finetuned))
print(np.mean(time_whisperx))

16.73553398188583
9.785396501780761
18.025972933615108
5.895717124785146
6.124618218791101
0.26086540375986406
